In [13]:
import os as os
from os import *
import random
import shutil

In [36]:
PATH = 'data'
train_root = f'{PATH}/train'
valid_root = f'{PATH}/valid'
classes = !ls {train_root}
classes_2 = os.listdir(train_root)
print(classes)
print(classes_2)

['BabyBibs', 'BabyHat', 'BabyPants', 'BabyShirt', 'PackageFart', 'womanshirtsleeve', 'womencasualshoes', 'womenchiffontop', 'womendollshoes', 'womenknittedtop', 'womenlazyshoes', 'womenlongsleevetop', 'womenpeashoes', 'womenplussizedtop', 'womenpointedflatshoes', 'womensleevelesstop', 'womenstripedtop', 'wrapsnslings']
['.DS_Store', 'wrapsnslings', 'womensleevelesstop', 'womencasualshoes', 'BabyBibs', 'womanshirtsleeve', 'womenplussizedtop', 'womenstripedtop', 'BabyHat', 'womenlazyshoes', 'womenknittedtop', 'womenpointedflatshoes', 'BabyShirt', 'womenlongsleevetop', 'womendollshoes', 'BabyPants', 'womenchiffontop', 'womenpeashoes', 'PackageFart']


In [32]:
def mv_to_valid(chosen_files, classname):
    for i in chosen_files:
        shutil.move(f'{train_root}/{classname}/{i}', f'{valid_root}/{classname}/{i}')

In [37]:
def train_val_split(val_percentage=0.2):
    random.seed(1)
    for classname in classes:
        os.makedirs(f'{valid_root}/{classname}', exist_ok=True)
        list_of_imgs = !ls "{train_root}/{classname}"
        random.shuffle(list_of_imgs)
        n_files_moved=int(len(list_of_imgs)*val_percentage)
        selected_files = [list_of_imgs[m] for m in range(n_files_moved)]
        mv_to_valid(selected_files, classname)

In [38]:
def mv_to_train(chosen_files, classname):
    for i in chosen_files:
        src = f'{valid_root}/{classname}/{i}'
        dst = f'{train_root}/{classname}/{i}'
        shutil.move(src, dst)
    
def put_back_to_train():
    for classname in classes:
        list_of_file_dirs = !ls "{PATH}/valid/{classname}"
        mv_to_train(list_of_file_dirs, classname)
    

In [26]:
put_back_to_train()

In [39]:
train_val_split()